In [136]:
import math
import numpy as np
from scipy import special
from skimage.draw.draw3d import ellipsoid
from skimage import measure
from IPython.core.debugger import set_trace

# %matplotlib notebook

#Sort according to Distance/Radius (Min To Max)
def qsort(distance,x_grid,y_grid,z_grid):
    startpoint = 0
    endpoint = len(distance)
    output = np.concatenate((distance,x_grid,y_grid,z_grid),axis=1).T
    if(startpoint < endpoint):
        flag = startpoint
        for j in range((startpoint+1),(endpoint)):
            if(output[0,startpoint]>output[0,j]):
                flag = flag+1
                temp = output[:,flag].copy()
                output[:,flag] = output[:,j]
                output[:,j] = temp
        temp = output[:,startpoint].copy()
        output[:,startpoint] = output[:,flag]
        output[:,flag] = temp
        output_0=np.matrix(output[:,startpoint:flag])
        output[:,startpoint:flag] = qsort(output_0[0,:].T,output_0[1,:].T,output_0[2,:].T,output_0[3,:].T)
        output_flag0=np.matrix(output[:,flag+1:endpoint])
        output[:,flag+1:endpoint] = qsort(output_flag0[0,:].T,output_flag0[0,:].T,output_flag0[0,:].T,output_flag0[0,:].T)
    return output

#Calculate Legendre
def legendre(n,X) :
    res = []
    for m in range(n+1):
        res.append(special.lpmv(m,n,X))
    return res

#Compute spherical harmonics
def spharm(L,M,THETA,PHI):
    if ((L==0) and (M ==0) and (THETA==0) and (PHI==0)):
        L=2
        M=1
        THETA = math.pi/4
        PHI = math.pi/4
        
    if L<M:
        raise ValueError('The ORDER (M) must be less than or eqaul to the DEGREE(L).')
    
    Lmn=np.array(legendre(L,np.cos(PHI)))
    
    if L!=0:
        Lmn=np.squeeze(Lmn[M,...])

    a1=((2*L+1)/(4*math.pi))
    a2=math.factorial(L-M)/math.factorial(L+M)
    C=np.sqrt(a1*a2)
    Ymn=C*Lmn*math.e**(M*THETA*1j)
    return Ymn

#Get Spherical descriptors
def spherical_descriptors_el(file,angle):
        
    verti_trans, faces_ellip,_,_ = measure.marching_cubes(file,0)
    verti_x = np.matrix(verti_trans[0].ravel())
    verti_y = np.matrix(verti_trans[1].ravel())
    verti_z = np.matrix(verti_trans[2].ravel())
    
    # rotated
    c, s = np.cos(angle), np.sin(angle)
    R = np.matrix([[c, -s], [s, c]])
    result = R* (np.array([verti_x,verti_y]))

    verti_x = result[0,:].T
    verti_y = result[1,:].T
    verti_z = verti_z.T
    
    #align coordinates
    verti_x = verti_x-np.min(verti_x)
    verti_y = verti_y-np.min(verti_y)
    verti_z = verti_z-np.min(verti_z)

    #normalize to 1 and rasterize to 2Rx2Rx2R voxel grid
    max_value = np.max([np.max(verti_x),np.max(verti_y),np.max(verti_z)])
    R = 32
    x1 = np.round(verti_x/max_value*(2*R-1))
    y1 = np.round(verti_y/max_value*(2*R-1))
    z1 = np.round(verti_z/max_value*(2*R-1))

    x_grid = []
    y_grid = []
    z_grid = []
    grid = np.zeros((2*R,2*R,2*R))
    n_points = len(x1)
    for j in range(n_points):
        if(grid[int(x1[j]),int(y1[j]),int(z1[j])]==0):
            #register
            grid[int(x1[j]),int(y1[j]),int(z1[j])]=1
            x_grid.append(x1[j])
            y_grid.append(y1[j])
            z_grid.append(z1[j])

    #get center of mass
    x_center =np.mean(x_grid)
    y_center =np.mean(y_grid)
    z_center =np.mean(z_grid)

    x_grid = np.array(x_grid).ravel() - x_center
    y_grid = np.array(y_grid).ravel() - y_center
    z_grid = np.array(z_grid).ravel() - z_center
    
    #scale and make the average distance to center of mass is R/2
    dist = np.sqrt((x_grid)**2 + (y_grid)**2 + (z_grid)**2)
    mean_dist = np.mean(dist)
    scale_ratio = (R/2)/mean_dist

    x_grid_sr = x_grid * scale_ratio
    y_grid_sr = y_grid * scale_ratio
    z_grid_sr = z_grid * scale_ratio

    final_dist = np.sqrt((x_grid_sr)**2 + (y_grid_sr)**2 + (z_grid_sr)**2)
    distance=np.matrix(final_dist).T
    x_grid_val=np.matrix(x_grid_sr).T
    y_grid_val=np.matrix(y_grid_sr).T
    z_grid_val=np.matrix(z_grid_sr).T

    # qsort function
    output = qsort(distance,x_grid_val,y_grid_val,z_grid_val)

    output1 = np.array(output).T
    dist_vector = output1[:,0]
    x     = output1[:,1]
    y     = output1[:,2]
    z     = output1[:,3]
    
    #Get phi value
    phi   = np.arctan2(output1[:,2],output1[:,1])
    
    #Get theta value
    aa= z/dist_vector
    theta = np.arccos(aa)
    max_l = 16
    max_r = 32
    sph = np.zeros((max_r,max_l))
    
    #Get shape descriptors
    for idx_n in range(0,len(dist_vector)+1,100):
        idx_r = math.ceil(dist_vector[idx_n])
        for idx_l in range(0,(max_l)):
            Y_ml = 0
            for idx_m in range(-idx_l,idx_l+1):
                if(idx_m>=0):
                    Y_ml = Y_ml+spharm(idx_l,idx_m,theta[idx_n],phi[idx_n])
                else:
                    Y_temp = spharm(idx_l,-idx_m,theta[idx_n],phi[idx_n])
                    Y_ml = Y_ml+(-1)**(-idx_m) * np.conj(Y_temp)
            F_lr = Y_ml
            sph[idx_r-1,idx_l]=sph[idx_r-1,idx_l] + abs(F_lr) ** 2
    sph_des=np.sqrt(sph)
    return sph_des

file =ellipsoid(5,3,4,spacing=(1,1,1),levelset=True)
angle = math.pi/6
desc_sph = spherical_descriptors_el(file,angle)
#print(desc_sph)